In [5]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urlparse, parse_qs
import re
import pandas as pd
import numpy as np
import time

# Delhi – MUMBAI  holidays

In [10]:
urls = [
    "https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


Processed https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure: 15 total rows collected so far
Processed https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct: 30 total rows collected so far
Processed https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure: 45 total rows collected so far
Processed https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-dir

In [11]:
df1 = pd.DataFrame(all_data)
df1

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR29,384.18",Air India,DEL,BOM,25 Dec,2h 10m,Direct,Business
1,"INR29,382.40",Air India,DEL,BOM,25 Dec,2h 25m,Cheapest direct,Business
2,"INR37,417.30",Air India,DEL,BOM,25 Dec,2h 05m,Direct,Business
3,"INR29,384.18",Air India,DEL,BOM,25 Dec,2h 15m,Direct,Business
4,"INR29,384.18",Air India,DEL,BOM,25 Dec,2h 15m,Direct,Business
...,...,...,...,...,...,...,...,...
84,"INR54,571.37",Air India,DEL,BOM,1 Jan,2h 10m,Direct,Business
85,"INR37,308.67",Air India,DEL,BOM,1 Jan,3h 45m,Direct,Business
86,"INR54,574.93",Air India,DEL,BOM,1 Jan,2h 25m,Direct,Business
87,"INR54,574.93",Air India,DEL,BOM,1 Jan,2h 25m,Direct,Business


# Delhi – MUMBAI  holidays

In [23]:
urls = [
    "https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct"
]

# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


Processed https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure: 15 total rows collected so far
Processed https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct: 30 total rows collected so far
Processed https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure: 45 total rows collected so far
Processed https://flights.booking.com/flights/DEL.AIRPORT-BOM.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BOM.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-dir

In [24]:
df2=pd.DataFrame(all_data)
df2

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR8,243.96",Air India,DEL,BOM,1 Jan,2h 05m,Direct,Economy
1,"INR8,142.31",Air India,DEL,BOM,1 Jan,2h 15m,Cheapest direct,Economy
2,"INR8,744.18",Air India,DEL,BOM,1 Jan,2h 10m,Direct,Economy
3,"INR8,744.18",Air India,DEL,BOM,1 Jan,2h 10m,Direct,Economy
4,"INR8,744.18",Air India,DEL,BOM,1 Jan,2h 10m,Direct,Economy
...,...,...,...,...,...,...,...,...
85,"INR7,465.20",IndiGo,DEL,BOM,26 Jan,2h 20m,Direct,Economy
86,"INR7,535.72",IndiGo,DEL,BOM,26 Jan,2h 20m,Direct,Economy
87,"INR7,555.66",Akasa Air,DEL,BOM,26 Jan,2h 20m,Direct,Economy
88,"INR8,420.07",Air India,DEL,BOM,26 Jan,2h 10m,Direct,Economy


# Banglore – Kolkata holidays

In [34]:
urls = [
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct"
]

# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


Processed https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure: 15 total rows collected so far
Processed https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct: 30 total rows collected so far
Processed https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure: 42 total rows collected so far
Processed https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-dir

In [36]:
df3 = pd.DataFrame(all_data)
df3

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR32,679.12",Air India,BLR,CCU,25 Dec,2h 20m,Cheapest direct,Business
1,"INR32,679.12",Air India,BLR,CCU,25 Dec,2h 25m,Direct,Business
2,"INR32,679.12",Air India,BLR,CCU,25 Dec,2h 35m,Direct,Business
3,"INR60,336.00",Air India,BLR,CCU,25 Dec,6h,Direct,Business
4,"INR61,578.00",Air India,BLR,CCU,25 Dec,6h 55m,1 stop,Business
...,...,...,...,...,...,...,...,...
81,"INR71,826.00",Air India,BLR,CCU,1 Jan,11h 30m,1 stop,Business
82,"INR100,106.00",Air India,BLR,CCU,1 Jan,7h 25m,1 stop,Business
83,"INR68,174.20",Air India,BLR,CCU,1 Jan,13h 10m,1 stop,Business
84,"INR74,401.87",Air India,BLR,CCU,1 Jan,12h,1 stop,Business


# Banglore – Kolkata holidays

In [51]:
urls = [
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct"
]

# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


Processed https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure: 11 total rows collected so far
Processed https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct: 24 total rows collected so far
Processed https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure: 39 total rows collected so far
Processed https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-dir

In [53]:
df4 = pd.DataFrame(all_data)
df4

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR7,814.98",IndiGo,BLR,CCU,1 Jan,2h 25m,Cheapest direct,Economy
1,"INR7,830.94",IndiGo,BLR,CCU,1 Jan,2h 25m,Direct,Economy
2,"INR7,814.98",IndiGo,BLR,CCU,1 Jan,2h 30m,Direct,Economy
3,"INR7,818.46",IndiGo,BLR,CCU,1 Jan,2h 30m,Direct,Economy
4,"INR7,818.46",IndiGo,BLR,CCU,1 Jan,2h 30m,Direct,Economy
...,...,...,...,...,...,...,...,...
72,"INR6,863.94",IndiGo,BLR,CCU,26 Jan,6h,1 stop,Economy
73,"INR6,863.94",IndiGo,BLR,CCU,26 Jan,6h,1 stop,Economy
74,"INR8,335.92",IndiGo,BLR,CCU,26 Jan,4h 55m,1 stop,Economy
75,"INR6,487.55",IndiGo,BLR,CCU,26 Jan,7h 05m,1 stop,Economy


# Chennai – Hyderabad Holidays

In [63]:
urls = [
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


Processed https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure: 15 total rows collected so far
Processed https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct: 30 total rows collected so far
Processed https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure: 44 total rows collected so far
Processed https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-dir

In [65]:
df5 = pd.DataFrame(all_data)
df5

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR19,857.76",Air India,MAA,HYD,25 Dec,1h 20m,Cheapest direct,Business
1,"INR33,851.00",Air India,MAA,HYD,25 Dec,4h 50m,Direct,Business
2,"INR31,510.00",Air India,MAA,HYD,25 Dec,5h 45m,1 stop,Business
3,"INR45,324.00",Air India,MAA,HYD,25 Dec,4h 30m,1 stop,Business
4,"INR32,222.00",Air India,MAA,HYD,25 Dec,9h 45m,1 stop,Business
...,...,...,...,...,...,...,...,...
84,"INR54,682.00",Air India,MAA,HYD,1 Jan,16h,1 stop,Business
85,"INR70,683.58",Air India,MAA,HYD,1 Jan,11h 35m,1 stop,Business
86,"INR96,965.00",Air India,MAA,HYD,1 Jan,7h 50m,1 stop,Business
87,"INR69,718.00",Air India,MAA,HYD,1 Jan,12h 15m,1 stop,Business


# Chennai – Hyderabad Holidays

In [72]:
urls = [
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct"
]



# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


Processed https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure: 14 total rows collected so far
Processed https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct: 29 total rows collected so far
Processed https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure: 44 total rows collected so far
Processed https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-dir

In [74]:
df6 = pd.DataFrame(all_data)
df6

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR4,856.34",IndiGo,MAA,HYD,1 Jan,1h 15m,Cheapest direct,Economy
1,"INR4,856.34",IndiGo,MAA,HYD,1 Jan,1h 15m,Direct,Economy
2,"INR4,856.34",IndiGo,MAA,HYD,1 Jan,1h 15m,Direct,Economy
3,"INR5,301.62",IndiGo,MAA,HYD,1 Jan,1h 15m,Direct,Economy
4,"INR4,856.34",IndiGo,MAA,HYD,1 Jan,1h 25m,Direct,Economy
...,...,...,...,...,...,...,...,...
83,"INR9,285.15",IndiGo,MAA,HYD,26 Jan,3h 30m,1 stop,Economy
84,"INR9,134.12",IndiGo,MAA,HYD,26 Jan,3h 35m,1 stop,Economy
85,"INR9,133.35",IndiGo,MAA,HYD,26 Jan,3h 45m,1 stop,Economy
86,"INR8,312.83",Air India,MAA,HYD,26 Jan,4h 20m,1 stop,Economy


# Mumbai – Goa Holidays

In [79]:
urls = [
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct"
]



# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


Processed https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure: 14 total rows collected so far
Processed https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct: 14 total rows collected so far
Processed https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure: 29 total rows collected so far
Processed https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-dir

In [81]:
df7 = pd.DataFrame(all_data)
df7

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR24,925.43",Air India,BOM,GOX,25 Dec,1h 20m,Cheapest direct,Business
1,"INR53,567.00",Air India,BOM,GOX,25 Dec,4h 25m,Direct,Business
2,"INR38,111.00",Air India,BOM,GOX,25 Dec,7h 55m,1 stop,Business
3,"INR39,058.00",Air India,BOM,GOX,25 Dec,8h 30m,1 stop,Business
4,"INR38,111.00",Air India,BOM,GOX,25 Dec,16h 05m,1 stop,Business
5,"INR92,573.00",Air India,BOM,GOX,25 Dec,6h 15m,1 stop,Business
6,"INR100,469.00",Air India,BOM,GOX,25 Dec,5h 45m,1 stop,Business
7,"INR40,004.00",Air India,BOM,GOX,25 Dec,22h 30m,1 stop,Business
8,"INR118,410.00",Air India,BOM,GOX,25 Dec,8h 25m,1 stop,Business
9,"INR105,418.00",Air India,BOM,GOX,25 Dec,10h 30m,1 stop,Business


# Mumbai - Goa 

In [86]:
urls = [
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct"
]



# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


Processed https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure: 15 total rows collected so far
Processed https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct: 29 total rows collected so far
Processed https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure: 44 total rows collected so far
Processed https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-dir

In [88]:
df8 = pd.DataFrame(all_data)
df8

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR5,977.56",Air India,BOM,GOX,1 Jan,1h 15m,Cheapest direct,Economy
1,"INR8,267.09",Akasa Air,BOM,GOX,1 Jan,1h 20m,Direct,Economy
2,"INR10,834.21",SpiceJet,BOM,GOX,1 Jan,1h 15m,Direct,Economy
3,"INR15,923.47",Air India,BOM,GOX,1 Jan,1h 15m,Direct,Economy
4,"INR7,033.00",Air India,BOM,GOX,1 Jan,4h 25m,Direct,Economy
...,...,...,...,...,...,...,...,...
69,"INR8,594.00",Air India,BOM,GOX,26 Jan,19h 20m,1 stop,Economy
70,"INR19,118.00",Air India,BOM,GOX,26 Jan,7h 10m,1 stop,Economy
71,"INR8,412.27",Air India,BOM,GOX,26 Jan,21h 55m,1 stop,Economy
72,"INR8,195.91",Air India,BOM,GOX,26 Jan,25h 10m,1 stop,Economy


# Delhi Banglore

In [95]:
urls = [
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


Processed https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure: 15 total rows collected so far
Processed https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct: 30 total rows collected so far
Processed https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure: 45 total rows collected so far
Processed https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-dir

In [97]:
df9 = pd.DataFrame(all_data)
df9

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR34,572.61",Air India,DEL,BLR,25 Dec,2h 50m,Direct,Business
1,"INR31,983.00",Air India,DEL,BLR,25 Dec,9h 30m,1 stop,Business
2,"INR34,572.61",Air India,DEL,BLR,25 Dec,2h 50m,Cheapest direct,Business
3,"INR45,706.64",Air India,DEL,BLR,25 Dec,2h 40m,Direct,Business
4,"INR39,667.57",Air India,DEL,BLR,25 Dec,2h 50m,Direct,Business
...,...,...,...,...,...,...,...,...
85,"INR55,287.00",Air India,DEL,BLR,1 Jan,4h 50m,1 stop,Business
86,"INR35,546.00",Air India,DEL,BLR,1 Jan,8h 50m,1 stop,Business
87,"INR53,026.00",Air India,DEL,BLR,1 Jan,5h 35m,1 stop,Business
88,"INR53,858.00",Air India,DEL,BLR,1 Jan,5h 30m,1 stop,Business


# Delhi Banglore

In [102]:
urls = [
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


Processed https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure: 13 total rows collected so far
Processed https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct: 27 total rows collected so far
Processed https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure: 42 total rows collected so far
Processed https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-dir

In [104]:
df10 = pd.DataFrame(all_data)
df10

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR11,772.07",Air India,DEL,BLR,1 Jan,2h 50m,Cheapest direct,Economy
1,"INR19,474.47",SpiceJet,DEL,BLR,1 Jan,2h 35m,Direct,Economy
2,"INR11,772.07",Air India,DEL,BLR,1 Jan,2h 50m,Direct,Economy
3,"INR12,538.73",IndiGo,DEL,BLR,1 Jan,2h 40m,Direct,Economy
4,"INR12,208.90",IndiGo,DEL,BLR,1 Jan,2h 50m,Direct,Economy
...,...,...,...,...,...,...,...,...
79,"INR10,682.76",IndiGo,DEL,BLR,26 Jan,2h 45m,Direct,Economy
80,"INR10,631.79",Air India,DEL,BLR,26 Jan,2h 50m,Direct,Economy
81,"INR10,631.79",Air India,DEL,BLR,26 Jan,2h 50m,Direct,Economy
82,"INR10,240.66",Akasa Air,DEL,BLR,26 Jan,3h 05m,Direct,Economy


# Kolkata – Guwahathi Holidays

In [110]:
urls = [
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


Processed https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure: 4 total rows collected so far
Processed https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct: 4 total rows collected so far
Processed https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure: 19 total rows collected so far
Processed https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-25&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direc

In [112]:
df11 = pd.DataFrame(all_data)
df11

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR27,008.95",Air India,CCU,GAU,25 Dec,1h 15m,Cheapest direct,Business
1,"INR86,947.00",Air India,CCU,GAU,25 Dec,6h 50m,Direct,Business
2,"INR92,543.00",Air India,CCU,GAU,25 Dec,15h 40m,1 stop,Business
3,"INR107,756.00",Air India,CCU,GAU,25 Dec,15h 10m,1 stop,Business
4,"INR10,975.83",IndiGo,CCU,GAU,25 Dec,1h 10m,Cheapest direct,Economy
5,"INR9,637.13",IndiGo,CCU,GAU,25 Dec,6h 50m,Direct,Economy
6,"INR11,078.91",IndiGo,CCU,GAU,25 Dec,1h 10m,1 stop,Economy
7,"INR11,794.48",IndiGo,CCU,GAU,25 Dec,1h 10m,Direct,Economy
8,"INR12,599.89",IndiGo,CCU,GAU,25 Dec,1h 10m,Direct,Economy
9,"INR12,702.97",IndiGo,CCU,GAU,25 Dec,1h 10m,Direct,Economy


# Kolkata – Guwahathi Holidays

In [119]:
urls = [
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


Processed https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure: 15 total rows collected so far
Processed https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-01&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-direct: 30 total rows collected so far
Processed https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure: 37 total rows collected so far
Processed https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-26&sort=BEST&travelPurpose=leisure&page=2&aid=304142&label=flights-booking-dir

In [121]:
df12 = pd.DataFrame(all_data)
df12

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR5,570.60",Air India,CCU,GAU,1 Jan,1h 15m,Cheapest direct,Economy
1,"INR5,969.87",IndiGo,CCU,GAU,1 Jan,1h 10m,Direct,Economy
2,"INR6,607.79",IndiGo,CCU,GAU,1 Jan,1h 10m,Direct,Economy
3,"INR6,813.94",IndiGo,CCU,GAU,1 Jan,1h 10m,Direct,Economy
4,"INR7,003.65",IndiGo,CCU,GAU,1 Jan,1h 10m,Direct,Economy
...,...,...,...,...,...,...,...,...
62,"INR7,049.26",IndiGo,CCU,GAU,26 Jan,22h 40m,1 stop,Economy
63,"INR7,369.52",Akasa Air,CCU,GAU,26 Jan,21h 50m,1 stop,Economy
64,"INR11,690.00",IndiGo,CCU,GAU,26 Jan,14h 25m,1 stop,Economy
65,"INR7,804.57",Air India,CCU,GAU,26 Jan,25h 50m,1 stop,Economy


In [123]:
holidays_data = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12])

In [125]:
holidays_data

,Price,Airline,Departure City,Arrival City,Travel Date,Duration,Stops,Cabin Class
0,"INR29,384.18",Air India,DEL,BOM,25 Dec,2h 10m,Direct,Business
1,"INR29,382.40",Air India,DEL,BOM,25 Dec,2h 25m,Cheapest direct,Business
2,"INR37,417.30",Air India,DEL,BOM,25 Dec,2h 05m,Direct,Business
3,"INR29,384.18",Air India,DEL,BOM,25 Dec,2h 15m,Direct,Business
4,"INR29,384.18",Air India,DEL,BOM,25 Dec,2h 15m,Direct,Business
...,...,...,...,...,...,...,...,...
62,"INR7,049.26",IndiGo,CCU,GAU,26 Jan,22h 40m,1 stop,Economy
63,"INR7,369.52",Akasa Air,CCU,GAU,26 Jan,21h 50m,1 stop,Economy
64,"INR11,690.00",IndiGo,CCU,GAU,26 Jan,14h 25m,1 stop,Economy
65,"INR7,804.57",Air India,CCU,GAU,26 Jan,25h 50m,1 stop,Economy


In [129]:
holidays_data.to_csv(r"H:\innomatics\EDA Project\final_holidays_data.csv")